In [30]:
import pandas as pd
import pyodbc

In [31]:
import pickle

# Load the model from a file
with open('model.pkl', 'rb') as file:
    model = pickle.load(file)


In [32]:
serverName = "datasolve.database.windows.net"
databaseName = "DataSolveDB"
username = "sqladmin"
password = "AlfaBravo2024"

In [33]:
connection_string = f'DRIVER={{SQL Server}};SERVER='+serverName+';DATABASE='+databaseName+';UID='+username+';PWD='+ password
 
cnxn = pyodbc.connect(connection_string)

In [34]:
# Query to obtain data from the SQL view
query = "SELECT * FROM [ml].[Death_GHDX]"

# Getting results
df = pd.read_sql(query, cnxn)

In [35]:
pred = model.predict(X)
pred

array([0.21191466, 0.15282784, 0.6270506 , ..., 0.9431696 , 0.14612849,
       0.1728039 ], dtype=float32)

In [36]:
result_df = pd.DataFrame({
    'Val': df['Val'],
    'Val_pred': pred,
    'year' : df['year']
})
result_df.head(100)

,Val,Val_pred,year
0,0.25,0.21,1990
1,0.10,0.15,1990
2,0.48,0.63,1990
3,0.14,0.07,1990
4,0.05,-0.02,1990
...,...,...,...
95,0.49,0.53,1990
96,0.37,0.43,1990
97,1.15,1.37,1990
98,0.32,0.27,1990


In [46]:
result_df['error'] = result_df['Val'] - pred

In [47]:
result_df.to_csv('result.csv', index=False)


In [48]:
result_df

,Val,Val_pred,year,error
0,0.25,0.21,1990,0.03
1,0.10,0.15,1990,-0.05
2,0.48,0.63,1990,-0.14
3,0.14,0.07,1990,0.08
4,0.05,-0.02,1990,0.07
...,...,...,...,...
30895,0.16,0.25,2019,-0.09
30896,0.00,0.08,2019,-0.08
30897,0.71,0.94,2019,-0.23
30898,0.11,0.15,2019,-0.04


## PREDICTION ON SDI 2100


In [39]:
file_path = 'SDI_pred.csv'

In [40]:
df = pd.read_csv(file_path)

In [41]:
df_pred = df.copy()

#keep all the columns of df except "ASR" to perform the prediction
if 'ASR' in df.columns:
    df_pred.drop(columns=['ASR'], inplace=True)

df_pred

,location,year,SDI,cluster_sdi,Risk
0,Afghanistan,2020.00,0.34,Low,Tobacco
1,Afghanistan,2020.00,0.34,Low,Alcohol use
2,Afghanistan,2020.00,0.34,Low,Metabolic risks
3,Afghanistan,2020.00,0.34,Low,Dietary risks
4,Afghanistan,2020.00,0.34,Low,Low physical activity
...,...,...,...,...,...
82210,Zimbabwe,2100.00,0.50,Low-middle,Tobacco
82211,Zimbabwe,2100.00,0.50,Low-middle,Alcohol use
82212,Zimbabwe,2100.00,0.50,Low-middle,Metabolic risks
82213,Zimbabwe,2100.00,0.50,Low-middle,Dietary risks


In [42]:
# load the model
with open('model.pkl', 'rb') as model_file:
    model = pickle.load(model_file)

predictions = model.predict(df_pred)

In [43]:
df['ASR_predicted'] = predictions
df

,location,year,SDI,cluster_sdi,Risk,ASR_predicted
0,Afghanistan,2020.00,0.34,Low,Tobacco,0.33
1,Afghanistan,2020.00,0.34,Low,Alcohol use,0.30
2,Afghanistan,2020.00,0.34,Low,Metabolic risks,0.65
3,Afghanistan,2020.00,0.34,Low,Dietary risks,0.14
4,Afghanistan,2020.00,0.34,Low,Low physical activity,0.08
...,...,...,...,...,...,...
82210,Zimbabwe,2100.00,0.50,Low-middle,Tobacco,0.60
82211,Zimbabwe,2100.00,0.50,Low-middle,Alcohol use,0.45
82212,Zimbabwe,2100.00,0.50,Low-middle,Metabolic risks,1.42
82213,Zimbabwe,2100.00,0.50,Low-middle,Dietary risks,0.31


In [44]:
df.to_csv('predictions_future.csv', index=False)